In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin

import tensorflow as tf
from tensorflow.keras import layers, models

from HAR.transformers import CSIMinMaxScaler
from HAR.io import load_dataset

class CSIScaler(TransformerMixin, BaseEstimator):
    def __init__(self) -> None:
        super().__init__()
        self._mean = None

    def fit(self, *_, **__):
        return self

    def transform(self, X):
        X = np.swapaxes(X, 1, 2)
        n_samples, t_max, *_ = X.shape
        min_vec = np.min(X, axis=(0, 1))
        min_vec = np.expand_dims(min_vec, axis=(0, 1))
        X -= np.tile(min_vec, (n_samples, t_max, 1))
        max_vec = np.max(X, axis=(0, 1))
        max_vec = np.expand_dims(max_vec, axis=(0, 1))
        X /= np.tile(max_vec, (n_samples, t_max, 1))
        X = np.swapaxes(X, 1, 2)
        return X

def create_model(dim):
    model = models.Sequential()
    model.add(
        layers.Conv2D(
            8,
            (3, 4),
            kernel_initializer="he_normal",
            dilation_rate=(1, 1),
            activation="relu",
            input_shape=(dim[0], dim[1], 1),
        )
    )
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(
        layers.Conv2D(
            16,
            (3, 4),
            kernel_initializer="he_normal",
            dilation_rate=(1, 1),
            activation="relu",
        )
    )
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(
        layers.Conv2D(
            32,
            (5, 6),
            kernel_initializer="he_normal",
            dilation_rate=(2, 2),
            activation="relu",
        )
    )
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(5, kernel_initializer="he_normal"))  # Updated for 5 classes

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    return model

# Load data
X, y, nsamples, classnames, dim = load_dataset("/home/ravindu/Desktop/final/dataset/mat/HAR_complete.mat")
X = X.reshape((X.shape[0], *dim))
X = CSIScaler().fit_transform(X)

# Train-test-validation split
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_rem, y_rem, train_size=0.7, stratify=y_rem)

model = create_model(dim)
model.summary()

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

y_test_preds = model.predict(X_test)
y_test_preds = np.argmax(y_test_preds, axis=1)

print("\nTest Accuracy : {}".format(accuracy_score(y_test, y_test_preds)))
print("\nConfusion Matrix : ")
print(confusion_matrix(y_test, y_test_preds))
print("\nClassification Report :")
print(classification_report(y_test, y_test_preds, target_names=classnames))

plt.figure()
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
plt.legend(loc="lower right")
plt.show()

model.save("csi-sense.h5")

# LIME Analysis
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
from skimage.color import rgb2gray, gray2rgb

def prediction_for_lime(imgs):
    imgs_recons = rgb2gray(imgs)
    return model.predict(imgs_recons, verbose=0)

explainer = lime_image.LimeImageExplainer()

classwise_results = []
for i in range(len(classnames)):
    X_class = X_test[y_test == i]
    y_class = y_test[y_test == i]
    idx = np.random.randint(0, y_class.size)
    classwise_results.append(
        (
            explainer.explain_instance(gray2rgb(X_class[idx]), prediction_for_lime),
            idx,
            i,
        )
    )

for explanation, idx, cid in classwise_results:
    image, mask = explanation.get_image_and_mask(cid, num_features=dim[0] * dim[1])
    image = rgb2gray(image)
    plt.figure(figsize=(10, 5), constrained_layout=True)
    plt.subplot(1, 2, 1)
    plt.imshow(mark_boundaries(image, mask), aspect="auto")
    plt.subplot(1, 2, 2)
    plt.imshow(image, aspect="auto", cmap="hsv")
    plt.show()